In [ ]:
#Lets try to get it to auto update

In [19]:
import sqlite3
import csv
import os
import pandas as pd

# Specify the directory path
directory_path = r"J:\Python\ksm"

# Combine the directory path with the database file name
db_file_path = os.path.join(directory_path, "ksm.db")

# Connect to SQLite database
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

# Execute your SQL query to retrieve data with date formatted as 'YYYY-MM-DD'
query = """
    SELECT 
        strftime('%Y-%m-%d', date) as date,
        *
    FROM taxexempt 
    WHERE date >= '2023-12-28' AND date <= '2024-01-01';
"""
cursor.execute(query)

# Fetch the column names
column_names = [description[0] for description in cursor.description]

# Fetch the results
data = cursor.fetchall()

# Specify the output CSV file path
output_csv_path = os.path.join(directory_path, 'refunding', 'output_data.csv')

# Export data to CSV with column names
with open(output_csv_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # Write the header (column names)
    csv_writer.writerow(column_names)
    
    # Write the data
    csv_writer.writerows(data)

# Close the database connection
conn.close()


In [21]:
import os
import pandas as pd

# Replace 'path/to/taxexempt.csv' with the actual path to your CSV file
csv_path = os.path.join(directory_path, 'refunding', 'output_data.csv')
df = pd.read_csv(csv_path)

# Change the scale column to the corresponding year
df['year'] = 2023 + df['scale']

# Keep only relevant columns
df = df[['date', 'year', 'rating', 'value']]

# Convert the date column to the 'YYYY-MM-DD' format
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

# Pivot the DataFrame using pivot_table
df_pivot = df.pivot_table(index=['date', 'year'], columns='rating', values='value', aggfunc='mean')

# Reset the index to make 'year' a column again
df_pivot.reset_index(inplace=True)

# Reorder columns
df_pivot = df_pivot[['date', 'year', 'AAA', 'AA', 'A', 'BAA']]

# Specify the output Excel file path for the modified data
modified_excel_path = os.path.join(directory_path, 'refunding', 'modified_output_data.xlsx')

# Export modified data to Excel
df_pivot.to_excel(modified_excel_path, index=False)

# View the result
df_pivot.head()


rating,date,year,AAA,AA,A,BAA
0,2023-12-28,2024,2.67,2.68,2.74,3.13
1,2023-12-28,2025,2.52,2.53,2.63,2.99
2,2023-12-28,2026,2.37,2.39,2.49,2.84
3,2023-12-28,2027,2.29,2.31,2.43,2.80
4,2023-12-28,2028,2.28,2.31,2.43,2.84


In [ ]:
#So trying to add this part into excel kept corrupting my database
#Just going to have to take modified_output_data.csv and manually add this into the data